<a href="https://colab.research.google.com/github/agdiaz/public-b2b/blob/main/AlphaFold_Database_Fetch_and_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Unitprot Selector

import requests
import json
import urllib

unitprot_accession = 'O15552' #@param {type:"string"}

url = "https://www.alphafold.ebi.ac.uk/api/prediction/{accession}".format(accession=unitprot_accession)

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
if response.status_code == 200:
  record = json.loads(response.text)[0]
  cifUrl = record['cifUrl']
  pdbUrl = record['pdbUrl']
  sequence = record['uniprotSequence']
elif response.status_code == 404:
    raise Exception("Protein not found with Uniprot accession '{accession}'".format(accession=unitprot_accession))
else:
    raise Exception(response.raw)

In [ ]:
#@markdown Download files from the database 
import tqdm.notebook
import textwrap as tr

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
    with urllib.request.urlopen(cifUrl) as cif_stream:
        with open('{0}.cif'.format(unitprot_accession), 'wb') as cif_file:
              content = cif_stream.read()
              cif_file.write(content)
    pbar.update(33)
    
    with urllib.request.urlopen(pdbUrl) as pdb_stream:
        with open('{0}.pdb'.format(unitprot_accession), 'wb') as pdb_file:
              content = pdb_stream.read()
              pdb_file.write(content)
    pbar.update(33)
    
    with open('{0}.fa'.format(unitprot_accession), 'w') as fasta_file:
        fasta = ">{key}\n{seq}".format(key=unitprot_accession, seq=tr.fill(sequence, width=80))
        fasta_file.write(fasta)
    pbar.update(34)

  0%|          | 0/100 [elapsed: 00:00 remaining: ?]

In [ ]:
#@markdown Download files in zip format
from google.colab import files

output_dir = '/content/'

!zip -q {unitprot_accession}.zip {unitprot_accession}.cif {unitprot_accession}.pdb {unitprot_accession}.fa
files.download(f'{unitprot_accession}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>